In [ ]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from google.colab import drive
import shutil
from IPython.display import FileLink

drive.mount('/content/drive')

folder_path_positive = '/content/drive/MyDrive/BrainTumorDataset/TumorNegative'
folder_path_negative = '/content/drive/MyDrive/BrainTumorDataset/TumorPositive'


def load_and_resize_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        try:
            with Image.open(img_path) as img:
                img = img.convert('L')
                img = img.resize((108, 108))
                img_array = np.array(img)
                images.append(img_array)
        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")
    return images


positive_images = load_and_resize_images(folder_path_positive)
negative_images = load_and_resize_images(folder_path_negative)


x_train = np.array(positive_images + negative_images)


x_train = x_train.astype('float32') / 255.0


x_train_flat = x_train.reshape((x_train.shape[0], -1))


print("Shape of the array:", x_train_flat.shape)


original_dim = 108 * 108
intermediate_dim = 256
latent_dim = 100


inputs = tf.keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_var = layers.Dense(latent_dim)(h)


def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

encoder = tf.keras.Model(inputs, [z_mean, z_log_var, z])


decoder_h = layers.Dense(intermediate_dim, activation='relu')
decoder_mean = layers.Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

vae = tf.keras.Model(inputs, x_decoded_mean)


kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
mse_loss = tf.keras.losses.mean_squared_error(inputs, x_decoded_mean)
mse_loss *= original_dim
vae_loss = tf.keras.backend.mean(mse_loss + kl_loss)



vae.add_loss(vae_loss)
vae.compile(optimizer='adam')


elbo_value = vae.evaluate(x_train_flat, x_train_flat, batch_size=batch_size)
print("ELBO:", elbo_value)


num_epochs = 200
batch_size = 64

history = vae.fit(x_train_flat, epochs=num_epochs, batch_size=batch_size)


num_generated_images = 10
encoded_images = encoder.predict(x_train_flat[:num_generated_images])[2]
decoded_images = vae.predict(x_train_flat[:num_generated_images])

output_folder = '/content/generated_images'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i in range(num_generated_images):
    img_array = (decoded_images[i] * 255).astype(np.uint8)
    img = Image.fromarray(img_array.reshape(108, 108))
    img_path = os.path.join(output_folder, f'generated_image_{i + 1}.png')
    img.save(img_path)


shutil.make_archive('/content/generated_images', 'zip', output_folder)


FileLink(r'/content/generated_images.zip')

print("Encoder Summary:")
encoder.summary()

print("\nVAE Summary:")
vae.summary()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Shape of the array: (3140, 11664)
50/50 [==============================] - 0s 4ms/step - loss: 1443.8285
ELBO: 1443.8284912109375
Epoch 1/200
50/50 [==============================] - 2s 6ms/step - loss: 632.5043
Epoch 2/200
50/50 [==============================] - 0s 7ms/step - loss: 467.2396
Epoch 3/200
50/50 [==============================] - 0s 7ms/step - loss: 417.4544
Epoch 4/200
50/50 [==============================] - 0s 7ms/step - loss: 388.4678
Epoch 5/200
50/50 [==============================] - 0s 6ms/step - loss: 350.4893
Epoch 6/200
50/50 [==============================] - 0s 7ms/step - loss: 333.1875
Epoch 7/200
50/50 [==============================] - 0s 7ms/step - loss: 313.4080
Epoch 8/200
50/50 [==============================] - 0s 7ms/step - loss: 297.0102
Epoch 9/200
50/50 [==============================] - 0s 7ms/step - loss: 289.0472
Epo